<a href="https://colab.research.google.com/github/juanmmartinss/Inteligencia_Artificial/blob/main/JUAN_SAVIO_PROJ_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import glob
import numpy as np
from PIL import Image
import os
import gdown
import zipfile
import torch
import re
import math
import cv2
import csv
import pandas as pd
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

!pip install -q ultralytics
from ultralytics import YOLO

!pip install -q transformers
from transformers import AutoImageProcessor, AutoModelForDepthEstimation

___________________________
#**OBTER DADOS DO BARACCA DATASET**
___________________________

**Abaixo os dados (imagens e annotations) do Baracca Dataset são obtidos e extraídos para a pasta local do Colab.**

In [ ]:
url = 'https://drive.google.com/uc?id=13dmP9NpnvIGJ2-ym310rhAhzvvStrCPU'
gdown.download(url, output='/content/anthropometric_annotations.csv')
url = 'https://drive.google.com/uc?id=1TNxdQR07rcMTA156dJ_1Q37VIDJlbxqZ'
gdown.download(url, output='/content/data.zip')
with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data')

In [ ]:
# shutil.copy('/content/drive/MyDrive/IA_FINAL/anthropometric_annotations.csv', '/content')
# shutil.copytree('/content/drive/MyDrive/IA_FINAL/dataset', '/content/dataset')

**Abaixo o dataset é filtrado para extrair as imagens necessárias, copiando apenas as imagens RGB para uma única pasta. São obtidos dados dos 30 modelos, porém somente das cenas 002, 003 e 004 que são as relevantes.**

In [ ]:
!mkdir /content/dataset/images
!mkdir /content/dataset/images/train
!mkdir /content/dataset/images/val
!mkdir /content/dataset/images/test

def img_RGB(diretorio):
  for arquivo in os.listdir(diretorio):
    arquivo_path = os.path.join(diretorio, arquivo)
    if os.path.isfile(arquivo_path) and 'RGB' in arquivo:

      match = re.match(r'imgRGB_\d{3}_(\d{3})_\d{3}.png', arquivo)
      cena = match.group(1)
      if cena == '002' or cena == '003' or cena == '004':
        match = re.match(r'imgRGB_(\d{3})_\d{3}_\d{3}.png', arquivo)
        pessoa = match.group(1)
        pessoa = int(pessoa)
        if pessoa < 26:
          shutil.move(arquivo_path, '/content/dataset/images/train')
        elif pessoa > 27:
          shutil.move(arquivo_path, '/content/dataset/images/test')
        else:
          shutil.move(arquivo_path, '/content/dataset/images/val')

    elif os.path.isdir(arquivo_path):
      img_RGB(arquivo_path)

img_RGB('/content/data')
shutil.rmtree('/content/data')

**Abaixo é realizado um downscaling para 360x640 pois as imagens originais têm resolução muito alta (1080x1920).**

In [ ]:
listas = [
    glob.glob('/content/dataset/images/train/*.png'),
    glob.glob('/content/dataset/images/val/*.png'),
    glob.glob('/content/dataset/images/test/*.png')
    ]
for lista in listas:
  for path in lista:
    image = Image.open(path)
    w, h = image.size
    image = image.resize((w // 3, h // 3))
    image.save(path)

___________________________
#**GERAR DETECÇÕES E SEGMENTAÇÕES**
___________________________

**Abaixo é realizada a predição das imagens obtidas do Baracca Dataset com um modelo pré-treinado da arquitetura YOLOv8.**\
**As annotations de Bounding Box e coordenadas de segmentação são salvas para uso na construção do dataset sintético.**

In [ ]:
model = YOLO('yolov8n-seg.pt')
listas = [
    glob.glob('/content/dataset/images/train/*.png'),
    glob.glob('/content/dataset/images/val/*.png'),
    glob.glob('/content/dataset/images/test/*.png')
    ]
for lista in listas:
  for path in lista:
    image = Image.open(path)
    results = model.predict(source=image, classes=0, conf=0.2, save_txt=True)

**Abaixo é realizada a organização das labels no dataset, usando a mesma estrutura de pastas das imagens.**

In [ ]:
!mkdir /content/dataset/labels
!mkdir /content/dataset/labels/train
!mkdir /content/dataset/labels/val
!mkdir /content/dataset/labels/test

def img_labels(diretorio):
  for arquivo in os.listdir(diretorio):
    arquivo_path = os.path.join(diretorio, arquivo)
    if os.path.isfile(arquivo_path):

      match = re.match(r'imgRGB_\d{3}_(\d{3})_\d{3}.txt', arquivo)
      cena = match.group(1)
      if cena == '002' or cena == '003' or cena == '004':
        match = re.match(r'imgRGB_(\d{3})_\d{3}_\d{3}.txt', arquivo)
        pessoa = match.group(1)
        pessoa = int(pessoa)
        if pessoa < 26:
          shutil.move(arquivo_path, '/content/dataset/labels/train')
        elif pessoa > 27:
          shutil.move(arquivo_path, '/content/dataset/labels/test')
        else:
          shutil.move(arquivo_path, '/content/dataset/labels/val')

    elif os.path.isdir(arquivo_path):
      img_labels(arquivo_path)

img_labels('/content/runs/segment/predict/labels')
shutil.rmtree('/content/runs')

**Abaixo é obtido o arquivo YAML para que o treinamento entenda a estrutura do dataset.**

In [ ]:
url = 'https://drive.google.com/uc?id=1R-Yh2LWiogV_VnAVCm3dZO6-ZCeEqdkT'
gdown.download(url, output='/content/dataset/data.yaml')

___________________________
#**GERAR MAPAS DE PROFUNDIDADE**
___________________________

**Abaixo são gerados os mapas de profundidade para as imagens obtidas do Baracca Dataset. É obtida e salva uma matriz de mesma dimensão da imagem para uso na construção do dataset sintético, onde para cada pixel tem-se um valor de profundidade em metros.**

In [ ]:
!mkdir /content/depth
!mkdir /content/depth/train
!mkdir /content/depth/val
!mkdir /content/depth/test

image_processor = AutoImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = AutoModelForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

listas = [
    glob.glob('/content/dataset/images/train/*.png'),
    glob.glob('/content/dataset/images/val/*.png'),
    glob.glob('/content/dataset/images/test/*.png')
    ]
i = 0
for lista in listas:
  for path in lista:
    i += 1
    print(f'{i} - {path}')
    image = Image.open(path)
    pixel_values = image_processor(image, return_tensors="pt").pixel_values

    with torch.no_grad():
        outputs = model(pixel_values)
        predicted_depth = outputs.predicted_depth

    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=image.size[::-1],
        mode="bicubic",
        align_corners=False,
    ).squeeze()
    output = prediction.numpy()

    path = path.replace('/content/dataset/images/', '').replace('.png', '')
    np.save(f'/content/depth/{path}.npy', output)

shutil.move('/content/depth', '/content/dataset')

In [ ]:
# shutil.copytree('/content/drive/MyDrive/IA_FINAL/depth', '/content/dataset/depth')

___________________________
#**FINETUNING DO RECONHECIMENTO DE PESSOAS**
___________________________

**Abaixo é realizado o finetuning do modelo de detecção e segmentação de pessoas utilizando os dados obtidos, separados em conjuntos de treino, validação e teste (como definido em data.yaml).**

In [ ]:
model_detect = YOLO('yolov8n-seg.pt')
model_detect.train(data='/content/dataset/data.yaml', epochs=20, imgsz=(360, 640),
                   patience=5, exist_ok=True)

**Abaixo é carregado o melhor modelo de detecção que foi salvo previamente.**

In [ ]:
url = 'https://drive.google.com/uc?id=1jlFChfVfWbmOdVnKU-mWg-h0xNxx4feS'
gdown.download(url, output='/content/best.pt')
model_detect = YOLO('/content/best.pt')

___________________________
#**TESTE DAS ESTIMATIVAS DE ALTURA**
___________________________

**Abaixo é realizado o teste das estimativas de altura em uma cena para todos os modelos, visto que se tem a altura real de todos para fins de comparação.**

In [ ]:
image_processor = AutoImageProcessor.from_pretrained('vinvino02/glpn-nyu')
model = AutoModelForDepthEstimation.from_pretrained('vinvino02/glpn-nyu')

In [ ]:
listas = [
    glob.glob('/content/dataset/images/train/*.png'),
    glob.glob('/content/dataset/images/val/*.png'),
    glob.glob('/content/dataset/images/test/*.png')
    ]

saida = open("/content/teste_altura.txt", "w")

for lista in listas:
  for path in lista:
    partes = path.split('/')
    arquivo = partes[5]
    valores = arquivo.split('_')
    id = int(valores[1])
    # cena = valores[2]
    # if(cena != '003'):
    #   continue

    results = model_detect.predict(source=path, classes=0, conf=0.2, max_det=1)

    # obter coordenadas em pixels
    coords = results[0].masks[0].xy

    xy_head = coords[0][0]
    for coord in coords[0]:
      if coord[1] < xy_head[1]:
          xy_head = coord

    xy_feet = coords[0][0]
    for coord in coords[0]:
      if coord[1] > xy_feet[1]:
          xy_feet = coord

    # obter coordenadas normalizadas
    coords = results[0].masks[0].xyn

    xy_head_norm = coords[0][0]
    for coord in coords[0]:
      if coord[1] < xy_head_norm[1]:
          xy_head_norm = coord

    xy_feet_norm = coords[0][0]
    for coord in coords[0]:
      if coord[1] > xy_feet_norm[1]:
          xy_feet_norm = coord

    # ajustar coordenadas para evitar acertar o background
    xy_head[1] += 5
    xy_feet[1] -= 5



    image = Image.open(path)
    pixel_values = image_processor(image, return_tensors="pt").pixel_values

    with torch.no_grad():
        outputs = model(pixel_values)
        predicted_depth = outputs.predicted_depth

    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=image.size[::-1],
        mode='bicubic',
        align_corners=False,
    ).squeeze()
    output = prediction.numpy()

    depth_head = round(output[int(xy_head[1])][int(xy_head[0])] * 100, 2)
    depth_feet = round(output[int(xy_feet[1])][int(xy_feet[0])] * 100, 2)
    depth = round((depth_head + depth_feet) / 2, 2)

    f = 55
    altura = math.sqrt(math.pow((xy_head[0] * depth_head - xy_feet[0] * depth_feet) / f, 2) +
                        math.pow((xy_head[1] * depth_head - xy_feet[1] * depth_feet) / f, 2) +
                        math.pow(depth_head - depth_feet, 2))
    altura = round(altura / 10, 2)

    linha = id # id do modelo
    coluna = 3 # altura em cm
    altura_real = None
    with open('/content/anthropometric_annotations.csv', 'r') as arq:
            reader = csv.reader(arq)
            for i, row in enumerate(reader):
                if i == linha:
                    altura_real = row[coluna]

    print(f'Distância estimada: {depth}cm, Altura (estimada | real): {altura}cm | {altura_real}cm')
    saida.write(str(depth) + ', ' + str(altura) + ', ' + str(altura_real) + '\n')

saida.close()


image 1/1 /content/dataset/images/train/imgRGB_025_004_003.png: 640x384 1 person, 75.9ms
Speed: 5.7ms preprocess, 75.9ms inference, 38.1ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /content/dataset/images/train/imgRGB_017_004_004.png: 640x384 1 person, 9.0ms
Speed: 1.1ms preprocess, 9.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)


Distância estimada: 281.86cm, Altura (estimada | real): 176.0cm | 160.0cm



image 1/1 /content/dataset/images/train/imgRGB_004_002_001.png: 640x384 1 person, 9.5ms
Speed: 1.5ms preprocess, 9.5ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 384)


Distância estimada: 267.2cm, Altura (estimada | real): 193.72cm | 184.0cm



image 1/1 /content/dataset/images/train/imgRGB_013_004_000.png: 640x384 1 person, 7.9ms
Speed: 1.1ms preprocess, 7.9ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)


...

Distância estimada: 209.99cm, Altura (estimada | real): 185.26cm | 187.0cm



image 1/1 /content/dataset/images/test/imgRGB_030_004_009.png: 640x384 1 person, 13.2ms
Speed: 1.4ms preprocess, 13.2ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 384)


Distância estimada: 172.5cm, Altura (estimada | real): 183.25cm | 187.0cm



image 1/1 /content/dataset/images/test/imgRGB_030_003_006.png: 640x384 1 person, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)


Distância estimada: 275.3cm, Altura (estimada | real): 175.63cm | 173.0cm
Distância estimada: 219.62cm, Altura (estimada | real): 167.14cm | 173.0cm
